# Тестовое задание

In [123]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import string

In [124]:
count_vect = CountVectorizer(max_df = 0.7, min_df = 0.05, ngram_range=(1, 2))
tfidf_vect = TfidfVectorizer(max_df = 0.7, min_df = 0.05, ngram_range=(1, 2))
porter_stemmer = nltk.stem.PorterStemmer()
tok = count_vect.build_tokenizer()
clf = LogisticRegression(solver='liblinear')

## Cкачиваем файл с данными

In [451]:
#!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
#или
#!wget https://drive.google.com/open?id=1DwwTyIn5zW7KTxp4bONHtKNa6DlT9PUj

## Если необходимо, преобразуем JSON к CSV

In [452]:
#df = pd.read_json ('reviews_Electronics_5.json', lines=True )
#df.to_csv ('Electronics_5.csv', index = None)

# Предобработка

In [31]:
dat = pd.read_csv("Electronics_5.csv")[["asin","reviewText", "overall"]]

## Добавляем бинрную оценку

In [32]:
dat["bin"] = dat.overall > 3

## Отрежем кусок даты

In [125]:
dat_t, dat = dat, dat[0:1001]

## Натравим на необработанные данные

In [126]:
dat_count_deft = count_vect.fit_transform(dat.reviewText.apply(lambda x: np.str_(x)))
dat_tfidf_deft = tfidf_vect.fit_transform(dat.reviewText.apply(lambda x: np.str_(x)))


## Приводим к базовой форме

In [81]:
dat["words"] = list(map(lambda elem: (" ").join(map(lambda s: (porter_stemmer.stem(s)), elem)).lower().translate(str.maketrans('', '', string.punctuation)), map(lambda wrds: tok(wrds), dat.reviewText.apply(lambda x: np.str_(x)))))

/home/denis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Можно воспользоваться анализатором, который предоставляют векторайзеры

In [82]:
# anlz = count_vect.build_analyzer()
# dat.words = list(map(lambda s: (" ").join(anlz(s)), dat.reviewText))

## Натравим на обработанные данные

In [127]:
dat_count = count_vect.fit_transform(dat.words)
dat_tfidf = tfidf_vect.fit_transform(dat.words)

## Проверим, что получилось

In [128]:
dat

,asin,reviewText,overall,bin,words
0,0528881469,We got this GPS for my husband who is an (OTR)...,5,True,we got thi gp for my husband who is an otr ove...
1,0528881469,"I'm a professional OTR truck driver, and I bou...",1,False,profession otr truck driver and bought tnd 700...
2,0528881469,"Well, what can I say. I've had this unit in m...",3,False,well what can say ve had thi unit in my truck ...
3,0528881469,"Not going to write a long review, even thought...",2,False,not go to write long review even thought thi u...
4,0528881469,I've had mine for a year and here's what we go...,1,False,ve had mine for year and here what we got it t...
...,...,...,...,...,...
996,9983891212,What is there to say? I needed a cable to run...,5,True,what is there to say need cabl to run from the...
997,9983891212,I can find nothing wrong with this cable. It ...,5,True,can find noth wrong with thi cabl it work perf...
998,9983891212,Works perfectly at 1080p HD and in all other C...,5,True,work perfectli at 1080p hd and in all other ca...
999,9983891212,I was looking for a light weight cable to go f...,1,False,wa look for light weight cabl to go from small...


## Разобьем вектора на выборки

In [157]:
count_train_mtrx, count_test_mtrx, count_train_ans, count_test_ans = train_test_split(
    dat_count, dat.bin, test_size=0.2)
#dat_count.toarray()

In [158]:
tfidf_train_mtrx, tfidf_test_mtrx, tfidf_train_ans, tfidf_test_ans = train_test_split(
    dat_tfidf, dat.bin, test_size=0.2)
#dat_tfidf.toarray()

## Обучим модели и посмотрим их точность

In [159]:
clf.fit(count_train_mtrx, count_train_ans)
cout_pred = clf.predict(count_test_mtrx)
accuracy_score(cout_pred, count_test_ans)

0.7711442786069652

In [160]:
clf.fit(tfidf_train_mtrx, tfidf_train_ans)
tfidf_pred = clf.predict(tfidf_test_mtrx)
accuracy_score(tfidf_pred, tfidf_test_ans)

0.8009950248756219

In [161]:
param_grid = {
    'C': np.arange(0.5, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(dat_count, dat.bin)
print(search.best_params_)

{'C': 0.6, 'penalty': 'l1'}


In [162]:
accuracy_score(dat.bin, search.best_estimator_.predict(dat_count))

0.9270729270729271

In [163]:
param_grid = {
    'C': np.arange(0.5, 3, 0.1),
    'penalty': ['l1', 'l2']
}
search = GridSearchCV(clf, param_grid, cv=5)
search.fit(dat_tfidf, dat.bin)
print(search.best_params_)

{'C': 1.0999999999999999, 'penalty': 'l1'}


In [164]:
accuracy_score(dat.bin, search.best_estimator_.predict(dat_tfidf))

0.8261738261738262